# fMRI Problem Set 1

The following problem set will require you to (1) load and visualize (f)MRI data in several different ways, (2) create a design matrix capturing the experimental condition, and (3) use regression to model fMRI activity. Each of these problems builds on tools and ideas we've introduced in the in-class demo notebooks. Once you've finished the problem set and all of your answers are displayed below, click *File* > *Export Notebook As...* and select *Export Notebook to HTML*. Open the HTML file in a browser (e.g. Chrome), then save the webpage as a PDF (e.g. using *File* > *Print...* in Chrome on Mac). Check the PDF to make sure your work rendered properly, then submit the PDF to [Gradescope](https://www.gradescope.com/). We'll start by loading in some general-purpose Python modules, but you'll need to load additional modules to complete the problems (look to the demo notebooks for examples). 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Problem 1: Visualization

fMRI datasets are complex and noisy, so it's important to visualize your data at every stage of analysis. We'll begin by loading in the dataset from [Haxby et al., 2001](https://doi.org/10.1126/science.1063736) using [Nilearn](https://nilearn.github.io/) (you'll need to change `data_dir` if you're doing this problem set locally).

In [2]:
from nilearn import datasets

# If you're working on a local machine, change this path
data_dir = '/jukebox/PNI-classes/students/NEU502/2021/nilearn-data'

# Load the Haxby et al., 2001 data via Nilearn
haxby_dataset = datasets.fetch_haxby(data_dir=data_dir)

For the first set of exercises, we'll visualize (*a*) the T1-weighted anatomical image, (*b*) the EPI image averaged across time, (*c*) an EPI volume at time point 1312, and (*d*) a mask demarcating ventral temporal (VT) cortex.

In [3]:
# Visualize the T1-weighted anatomical image here:



In [4]:
# Visualize the mean EPI image here:



In [5]:
# Visualize EPI volume 1394 here:
# Hint: check out the other nilearn.image functions



In [6]:
# Visualize the VT mask overlad on the mean EPI here:



Next, we'll use [NiBabel](https://nipy.org/nibabel/) to directly load in the data as [NumPy](https://numpy.org/) arrays for manipulation in Python. First, we'll load in the VT mask as a boolean array. Inspect the shape of the functional data and mask, then apply the mask to the functional data to get an array containing EPI time series for only VT cortex. Next, we'll plot the mean time series in VT. Finally, without using Nilearn, create a "carpet plot" via [Power et al., 2017](https://doi.org/10.1016/j.neuroimage.2016.08.009) for VT data where the x-axis corresponds to time and the y-axis corresponds to voxels. Make sure to z-score each voxel's time series prior to plotting (e.g. using `zscore` from `scipy.stats`).

In [7]:
import nibabel as nib

# Use NiBabel to load functional data and VT mask
func_img = nib.load(haxby_dataset.func[0])
func = func_img.get_fdata()
mask_vt = nib.load(haxby_dataset.mask_vt[0]).get_fdata().astype(bool)

# Keep the VT mask voxel indices for later
ids_vt = np.where(mask_vt)

In [8]:
# Inspect shape of functional data and mask here:



In [9]:
# Plot the mean time series in VT cortex here:



In [10]:
# Plot carpet plot of VT data here:



## Problem 2: Design

In fMRI analysis, we usually assume that the BOLD signal is the result of a linear time-invariant (LTI) system ([Boynton et al., 1996](http://www.jneurosci.org/content/16/13/4207)); in other words, we assume (1) the shape of the hemodynamic response is constant across time, and (2) the responses to successive stimuli superpose linearly (additively). Assumption #2 justifies using convolution in generating the predicted BOLD signal from a set of stimulus events, $x$, and a hemodynamic response function, $h$. Conceptually, what convolution does is add the entire HRF shape starting wherever there is a non-zero entry.

In the [Haxby et al., 2001](https://doi.org/10.1126/science.1063736) study, participants were presented with images from 8 object categories (bottles, cats, chairs, faces, houses, scissors, scrambled images, and shoes) interspersed with periods of fixation (referred to as "rest" here). The TR in this study was 2.5 seconds. In a given run, a block of images from each of the 8 categories was presented one time. Each block was ~9 TRs long and contained multiple rapid presentations of images from a single category. A subject received 12 scanning runs. For the sake of simplicity, in this example, these runs are concatenated into a single time series 1452 TRs in duration.

In the following exercises, we'll start with the "face" condition. We'll (*a*) create a super-sampled time series of "boxcars" representing blocks of face images, then (*b*) convolve this time series with the SPM HRF, then (*c*) downsample this hypothesized BOLD time series to match the sampling rate (TR) of the fMRI data.

In [11]:
# Image acquition parameters
tr = 2.5
n_trs = 1452

# High-resolution (super-)sampling frequency
sfreq = .1
sst = np.arange(0, n_trs * tr, sfreq)

# Onset/offset timing for face blocks (or "events")
face_blocks = [(21, 29), (127, 135), (248, 256), (426, 434), (490, 498),
               (697, 705), (775, 783), (853, 861), (974, 982),
               (1124, 1132), (1302, 1310), (1394, 1402)]

# High-resolution (super-)sampled face onsets/offsets
face_sst = [(onset * tr, offset * tr)
            for onset, offset in face_blocks]

In [12]:
# Generate the boxcar time series for faces and plot here:



In [13]:
# Convolve the boxcar time series with the SPM HRF and plot here:



In [14]:
# Downsample and plot both time series here:



Now we'll use the time series of condition labels provided with the dataset to set to programmatically construct a single design matrix containing regressors for all conditions. We'll load in the time series of condition labels as a [pandas](https://pandas.pydata.org/) DataFrame, and extract all 8 category labels. Using code from above, create a loop that goes through each condition label and constructs a boxcar time series convolved with the SPM HRF, then stack these boxcar time series into a single design matrix. We've provided a helper function called `get_blocks` for converting the label time series from the dataset into our familiar onset/offset block structure. Finally, plot the design matrix, keeping in mind that you might need to transpose it or use `np.repeat` to make it more visible.

In [15]:
import pandas as pd

# Load in condition labels (i.e. "targets") as a pandas DataFrame
targets = pd.read_csv(haxby_dataset.session_target[0], sep=' ')

# Get 8 condition labels excluding "rest"
labels = [l for l in np.unique(targets['labels']).tolist()
          if l != 'rest']

In [16]:
# Function for converting targets to familiar block onsets/offsets
def get_blocks(targets, label):
    regressor = np.zeros(len(targets))
    regressor[targets['labels'] == label] = 1
    blocks = [(on + 1, off) for on, off in
              np.split(np.where(np.diff(regressor) != 0)[0], 12)]
    return blocks

# Make sure it returns the same thing as our previous face_blocks variable
assert(get_blocks(targets, 'face') == face_blocks)

In [17]:
# Create a loop that constructs design matrix here:



In [18]:
# Plot full design matrix here:



## Problem 3: Regression

Now that we have a set of predictors (or "regressors") capturing the object categories in our experimental design, we'll use a simple regression analysis to model the fMRI data. The goal is to discover which voxels are most responsive to certain object categories such as faces or houses. First, however, we need to create a set of confound regressors to capture noise in the data — in this case, we'll use detrending polynomials to account for slow signal drifts in each run. The full 1452-TR scan comprises 12 runs, each with 121 TRs. We'll need a 3rd-order polynomial expansion (i.e. a constant vector plus linear, quadratic, and cubic trends) per run. Each run should receive it's own set of detrending variables, with zeros for the other runs, meaning the final confound matrix should be shaped 1452 x 48. Creating this matrix a bit of a chore programming-wise, so we lay most of it out — but you need to fill in the polynomials.

In [19]:
from fmritools import legendre

# Set the runwise polynomial parameters
n_runs = 12
order = 3
run_trs = n_trs // n_runs

# Create a confound matrix with a block of polynomials for each run
confounds = [np.split(run, n_runs, axis=1) for run in
             np.split(np.zeros((n_trs, (order + 1) * n_runs)), n_runs)]
for run in np.arange(n_runs):
    
    confounds[run][run] = ### Fill in the Legendre polyomials here!!!

confounds = np.block(confounds)

SyntaxError: invalid syntax (<ipython-input-19-62d758ea1739>, line 13)

Next, stack this confound matrix alongside our regressors-of-interest to construct the final design matrix (referred to by convention as `X`). Plot this matrix.

In [ ]:
# Stack the regressors-of-interest with the confound matrix and plot here:



Now, we'll return to our functional time series in VT cortex from problem 1. This will serve as the `y` variable (i.e. dependent or outcome variable) in our regression model. (Note that you may need to transpose your `X` or `y` arrays to make sure they match along the time dimension). Now, use ordinary least-squares regression to fit the model and estimate the regression coefficients (referred to as "beta" coefficients or `b`). Matrix-multiply the design matrix `X` by the `b` coefficients to created a predicted time series for VT cortex. Visualize the actual mean activity across VT cortex and the mean predicted time series across VT cortex.

In [ ]:
# Run the regresssion to get beta coefficients here: 



In [ ]:
# Compute the predicted time series for VT cortex
# and plot the mean predicted time series alongside
# the actual predicted time series here:



Finally, we'll get the beta coefficients for our regressors-of-interest and plot them on the brain. First, preallocate a single volume of zeros matching the shape of our `func` data (i.e. 40, 64, 64). Next, get the `b` values corresponding to the face or house regressor. Then, use the `ids_vt` variable from above, which denotes the voxels in the full volume correspondig to the VT mask, to insert `b` values for VT cortex into the full volume. Use NiBabel's `Nifti1Image` with `func_img.affine` from above to create a NIfTI image, and supply this to one of Nilearn's `plot_stat_map` (using the mean EPI image as the `bg_img`).

In [ ]:
# Plot beta coefficients for the face regressor on the brain here:



In [ ]:
# Plot beta coefficients for the house regressor on the brain here:

